In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [4]:
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # 새로운 샘플이라고 생각합니다.

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 0.7941 - val_loss: 0.9327
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 1.3771 - val_loss: 3.8853
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 7.6263 - val_loss: 0.4491
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4366 - val_loss: 0.4281
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4328 - val_loss: 0.4015
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4154 - val_loss: 0.3862
Epoch 7/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3849 - val_loss: 0.3666
Epoch 8/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3681 - val_loss: 0.3577
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3653 - val_loss: 0.3563
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3611 - val_loss: 0.3488

In [5]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [6]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [7]:
model.compile(loss="mse", optimizer = keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Epoch 1/20


c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


363/363 [==============================] - 1s 2ms/step - loss: 2.1464 - val_loss: 0.9884
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 0.8300 - val_loss: 0.7452
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6881 - val_loss: 0.6667
Epoch 4/20
363/363 [==============================] - 1s 1ms/step - loss: 0.6411 - val_loss: 0.6289
Epoch 5/20
363/363 [==============================] - 1s 1ms/step - loss: 0.6118 - val_loss: 0.5998
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5892 - val_loss: 0.5774
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5715 - val_loss: 0.5586
Epoch 8/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5571 - val_loss: 0.5434
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5449 - val_loss: 0.5312
Epoch 10/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5349 - val_loss: 0.5199
Epoch 11/2

In [8]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # 표준 매개변수를 처리합니다.(예를 들면, name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
    
model = WideAndDeepModel()

In [9]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [10]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_36808/1709004121.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [14]:
from sklearn.metrics import mean_squared_error
keras_reg.fit(X_train, y_train, epochs=100, 
              validation_data=(X_valid, y_valid), 
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

y_pred = keras_reg.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)

print("테스트 세트 MSE:", mse_test)

# 새 데이터 예측
y_pred_new = keras_reg.predict(X_new)

Epoch 1/100


c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


363/363 [==============================] - 1s 2ms/step - loss: 1.6762 - val_loss: 1.3214
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 1.6054 - val_loss: 0.9983
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.6183 - val_loss: 0.5459
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5238 - val_loss: 0.5074
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4975 - val_loss: 0.4847
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4830 - val_loss: 0.4691
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4695 - val_loss: 0.4598
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4596 - val_loss: 0.4492
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4514 - val_loss: 0.4409
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4433 - val_loss: 0.4356
E

In [17]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_distribs = {
    "n_hidden" : [0, 1, 2, 3],
    "n_neurons" : np.arange(1, 100),
    "learning_rate" : reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100


c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 4ms/step - loss: 0.6698 - val_loss: 0.8507
Epoch 2/100
242/242 [==============================] - 1s 3ms/step - loss: 1.3626 - val_loss: 0.4187
Epoch 3/100
242/242 [==============================] - 1s 3ms/step - loss: 0.4898 - val_loss: 2.2100
Epoch 4/100
242/242 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 5/100
242/242 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 6/100
242/242 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 7/100
242/242 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 8/100
242/242 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 9/100
242/242 [==============================] - 1s 4ms/step - loss: nan - val_loss: nan
Epoch 10/100
242/242 [==============================] - 1s 5ms/step - loss: nan - val_loss: nan
Epoch 11/100
242/242 [=====================

c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 2s 4ms/step - loss: 0.8052 - val_loss: 0.6398
Epoch 2/100
242/242 [==============================] - 1s 3ms/step - loss: 0.9454 - val_loss: 0.9568
Epoch 3/100
242/242 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 4/100
242/242 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 5/100
242/242 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 6/100
242/242 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 7/100
242/242 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 8/100
242/242 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 9/100
242/242 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 10/100
242/242 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 11/100
121/121 [===========================

...

c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 3ms/step - loss: 0.7594 - val_loss: 0.5372
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4934 - val_loss: 0.4909
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4678 - val_loss: 0.5034
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5146 - val_loss: 0.5225
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4273 - val_loss: 0.5849
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4262 - val_loss: 0.6440
Epoch 7/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4165 - val_loss: 0.6912
Epoch 8/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4006 - val_loss: 0.7971
Epoch 9/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3965 - val_loss: 0.8702
Epoch 10/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4071 - val_loss: 0.9267
E

c:\Users\JunHuiLim\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [            nan -4.20370549e-01 -5.23111492e-01 -5.32235255e-01
 -1.55300347e+06 -3.96768818e-01 -3.29131107e-01             nan
 -3.28577558e-01             nan]
  warnings.warn(


363/363 [==============================] - 1s 2ms/step - loss: 1.8503 - val_loss: 0.7157
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.6568 - val_loss: 0.6274
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5954 - val_loss: 0.5751
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5500 - val_loss: 0.5306
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5123 - val_loss: 0.4963
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4820 - val_loss: 0.4698
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4600 - val_loss: 0.4471
Epoch 8/100
363/363 [==============================] - 1s 3ms/step - loss: 0.4443 - val_loss: 0.4333
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4332 - val_loss: 0.4224
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4244 - val_loss: 0.4156
E

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x0000016353361E80>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001635BDE1D60>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})